In [18]:
import requests
from bs4 import BeautifulSoup

urls = {
    "BIZNES" : {
        "url" : "https://businessinsider.com.pl/biznes",
        "pages" : 10,
        "query_type": "query"
        },
    "WYPADEK" : {
        "url" : "https://policja.pl/pol/aktualnosci?ida[6]=2140",
        "pages" : 10,
        "query_type": "query"
    },
    "POLITYKA": {
        "url" : "https://www.rp.pl/wydarzenia/kraj/polityka",
        "pages" : 20,
        "query_type" : "path"
    }
}


def get_page_data(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            return response.text
            print("Pobrano zawartość strony pomyślnie.")
        else:
            print(f"Błąd podczas pobierania strony. Kod statusu: {response.status_code}")
            exit()

    except requests.exceptions.RequestException as e:
        print(f"Wystąpił błąd połączenia: {e}")
        exit()

def get_business_main_data(url,page):
    if page > 1:
        url = url + f'?page={page}'
        
    content = get_page_data(url)
    soup = BeautifulSoup(content, 'html.parser')
    main = soup.find('main', class_='main')
    return main
    
def get_parsed_page_data(url,page,query_type):
    if query_type == 'query':
        if page > 1:
            url = url + f'?page={page}'
    if query_type == 'path':
        if page > 1:
            url = url + f'/{page}'
    
    content = get_page_data(url)
    soup = BeautifulSoup(content, 'html.parser')
    return soup

def get_container(content, elem_to_extract, class_name):
    return content.find(elem_to_extract,class_= class_name)
    
def extract_headers(element, header, class_name=''):
    return element.find_all(header,class_= class_name)

def extract_text(elements):
    text = []
    for element in elements:
        text.append(element.get_text(strip=True))
    return text

def get_business_headers(url_data):
    text_headers = []
    for page in range(1,url_data['pages']+1):
        content = get_parsed_page_data(url_data['url'],page,url_data['query_type'])
        main_container = get_container(content,'main','main')
        headers = extract_headers(main_container,'h2','item_title')
        text_headers.extend(extract_text(headers))
    return text_headers

def get_accidents_headers(url_data):
    text_headers = []
    for page in range(1,url_data['pages']+1):
        content = get_parsed_page_data(url_data['url'],page,url_data['query_type'])
        main_container = get_container(content,'div','okno')
        headers = extract_headers(main_container,'h3')
        text_headers.extend(extract_text(headers))
    return text_headers

def get_politics_headers(url_data):
    text_headers = []
    for page in range(1,url_data['pages']+1):
        content = get_parsed_page_data(url_data['url'],page,url_data['query_type'])
        headers = extract_headers(content,'h2','t--60')
        text_headers.extend(extract_text(headers))
    return text_headers

categories_headers = {
    "BIZNES" : [],
    "WYPADEK": [],
    "POLITYKA": []
}

categories_headers['BIZNES'] = get_business_headers(urls['BIZNES'])
categories_headers['WYPADEK'] = get_accidents_headers(urls['WYPADEK'])
categories_headers['POLITYKA'] = get_politics_headers(urls['POLITYKA'])

In [19]:
import pandas as pd
s = pd.Series(categories_headers)
df = s.explode().reset_index()
df.columns = ['Wydarzenie', 'Naglowek']
df.to_csv('wydarzenia.csv', index=False, encoding='utf-8')
print(df)

     Wydarzenie                                           Naglowek
0        BIZNES  Edukacja kluczem do osiągnięcia finansowego sp...
1        BIZNES  Nowy rozdział dla iRobot: dotychczasowy partne...
2        BIZNES  Forum Przedsiębiorczości - PARP4digital. Dokąd...
3        BIZNES  Powstała Koalicja na Rzecz Zdrowia Jamy Ustnej...
4        BIZNES       NEXT prezentuje limitowaną ofertę noworoczną
...         ...                                                ...
1010   POLITYKA  To koniec Platformy Obywatelskiej. W sobotę pr...
1011   POLITYKA  Michał Szułdrzyński: Rosyjska wojna hybrydowa ...
1012   POLITYKA  Czarzasty nie może być pewien, że zostanie mar...
1013   POLITYKA  Nowy sondaż partyjny: KO wraca na pozycję lide...
1014   POLITYKA  Donald Tusk ostro o Jarosławie Kaczyńskim: Pod...

[1015 rows x 2 columns]
